In [28]:
import pandas as pd
import numpy as np

In [29]:
# onedrive = 'F:/onedrive/'
onedrive = "C:/Users/jokit/OneDrive/"
path = f'{onedrive}Phd workshop/fmri/tadpole_challenge/'
data = pd.read_csv(f'{path}TADPOLE_D1_D2.csv')
D3 = pd.read_csv(f'{path}TADPOLE_D3.csv')
v1 = data['D1'].values
v2 = data['D2'].values
D1 = data.loc[v1 == 1,:]
D2 = data.loc[v2 == 1,:]

C:\Users\jokit\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (471,473,474,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,569,570,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,599,601,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,624,625,626,627,628,629,630,631,632,633,634,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723

In [30]:
cols = D3.columns.values
subD1 = D1[cols]
subD1['D'] = np.repeat('D1',len(subD1))

subD2 = D2[cols]
subD2['D'] = np.repeat('D2',len(subD2))

D3['D'] = np.repeat('D3',len(D3))

C:\Users\jokit\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jokit\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# Begin preprocessing the data

In [31]:
allData = pd.concat([subD1,subD2,D3],axis=0)
visc = allData['VISCODE'].values
age = allData['AGE'].values
for i,v in enumerate(visc):
    if v.startswith('m'):
        inc = float(v.split('m')[1])/12.
        age[i] += inc
allData['AGE'] = age

In [32]:
# infer the categorical features
import re
prefix = '(^\d+(\.\d+)?$)'
onehotMask = [False if re.match(prefix,str(x)) else True for x in allData.iloc[0,:].values]
contMask = [not x for x in onehotMask]

In [33]:
# predicting adas13
onehotCols = ['PTGENDER','PTETHCAT','PTRACCAT','PTMARRY']
contCols = ['AGE','PTEDUCAT','Hippocampus',\
            'WholeBrain','Entorhinal','Fusiform','MidTemp']

In [34]:
allData = allData.loc[~np.isnan(allData['ADAS13'].values),:]
# lots of the records are duplicated.
allData = allData.drop_duplicates(subset=['RID','AGE'])

## preprocessing for LGPR

In [9]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
contData = allData[contCols].values
onehotData = allData[onehotCols].values

scaler = StandardScaler()
imp = IterativeImputer(max_iter=30, random_state=0)
dataScaler = scaler.fit_transform(contData)
dataScaler = imp.fit_transform(dataScaler)

In [10]:
def categorizeFeature(x):
    idmap = {each:i for i,each in enumerate(set(x))}
    if len(idmap) > 1:
        return np.array([idmap[each] for each in x]).reshape(-1,1)
    return None

In [11]:
colnames = []
cateRes = None
for each in range(onehotData.shape[1]):
    if cateRes is None:
        r = categorizeFeature(onehotData[:,each])
        if r is not None:
            cateRes = r
            colnames.append(onehotCols[each])
    else:
        r = categorizeFeature(onehotData[:,each])
        if r is not None:
            cateRes = np.concatenate([cateRes,r],axis=1)
            colnames.append(onehotCols[each])
cateCols = np.arange(len(colnames))
colnames.extend(contCols)
colnames = np.array(colnames)
contCC = np.arange(len(cateCols),len(colnames))

## preprocessing for other algorithms

In [35]:
colnames = []
onehotModel = OneHotEncoder()
cateRes = onehotModel.fit_transform(onehotData.astype(str)).toarray()
for i in range(len(onehotModel.categories_)):
    colnames.extend([onehotCols[i]+'_'+str(x) for x in onehotModel.categories_[i]])
colnames.extend(contCols)
colnames = np.array(colnames)

In [36]:
idCol = allData['RID'].values
oidCol = allData['AGE'].values
X = np.concatenate([cateRes,dataScaler],axis=1)
y = allData['ADAS13'].values

In [20]:
# format a small dataset containing only several individuals
from collections import Counter
ct = Counter(idCol)
top = 50
targetIds = set([x[0] for x in ct.most_common()[:top]])
mask = [x in targetIds for x in idCol]
idCol = idCol[mask]
oidCol = oidCol[mask]
X = X[mask]
y = y[mask]

In [21]:
from collections import defaultdict

def splitTrainTest(ids, time, prop=.7, seed = 19):
    np.random.seed(seed)
    mask = np.array([np.random.uniform() < prop for x in ids])
    
    trainIdx, trainTime = ids[mask], time[mask]
    testIdx,testTime = ids[~mask],time[~mask]
    trainDict = defaultdict(list)
    testDict = defaultdict(list)
    
    for i,m in enumerate(mask):
        if m:
            trainDict[ids[i]].append((time[i],i))
        else:
            testDict[ids[i]].append((time[i],i))
    for k,v in trainDict.items():
        trainV = [x[0] for x in v]
        trainVid = [x[1] for x in v]
        tv = testDict.get(k,None)
        if tv is not None:
            testV = [x[0] for x in tv]
            testVid = [x[1] for x in tv]
            maxId = np.argmax(trainV)
            minId = np.argmin(testV)
            while trainV[maxId] > testV[minId]:
                tmp = trainV[maxId]
                trainV[maxId] = testV[minId]
                testV[minId] = tmp
                tmp = trainVid[maxId]
                trainVid[maxId] = testVid[minId]
                testVid[minId] = tmp
                maxId = np.argmax(trainV)
                minId = np.argmin(testV)
            trainDict[k] = [(x,y) for x,y in zip(trainV,trainVid)]
            testDict[k] = [(x,y) for x,y in zip(testV,testVid)]
    train = []
    test = []
    for k,v in trainDict.items():
        train.extend([x[1] for x in v])
    for k,v in testDict.items():
        test.extend([x[1] for x in v])
    return np.array(train), np.array(test)

def getIndvFixFeature(ids,x):
    dt = defaultdict(list)
    for i,k in enumerate(ids):
        dt[k].append(x[i])
    noChange = np.repeat(True,x.shape[1])
    for k,v in dt.items():
        v = np.array(v)
        for i in range(v.shape[1]):
            if np.sum(v[:,i] - v[0,i]) != 0:
                noChange[i] = False
    return noChange

from scipy import io
def generate(seed = 19,density = 0.7,name = 'tadpole'):
    trainIdx, testIdx = splitTrainTest(idCol, X[:,np.where(colnames == 'AGE')[0]].reshape(-1),density,seed)
    if name == 'tadpoleLGPR':
        io.savemat(f'../tadpoleLGPR_{seed}',{'trainId':idCol[trainIdx],'trainOid':oidCol[trainIdx],'trainX':X[trainIdx],'trainY':y[trainIdx],
                                  'testId':idCol[testIdx],'testOid':oidCol[testIdx],'testX':X[testIdx],'testY':y[testIdx],'colname':colnames,
                                        'cateCols':cateCols, 'contCols':contCC})
    else:
        io.savemat(f'../tadpole_{seed}',{'trainId':idCol[trainIdx],'trainOid':oidCol[trainIdx],'trainX':X[trainIdx],'trainY':y[trainIdx],
                                  'testId':idCol[testIdx],'testOid':oidCol[testIdx],'testX':X[testIdx],'testY':y[testIdx],'colname':colnames})

In [27]:
generate(seed=14,name='tadpole')